<a href="https://colab.research.google.com/github/LeeGitaek/2020_AI_Class/blob/master/MNIST_CNN_LeNet5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [47]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [48]:
#parameters
learning_rate = 0.9
training_epochs = 50
batch_size = 256 

In [49]:
#MNIST 영상이 28 * 28로 가공되어 있어, Original 입력 사이즈 32*32로 맞추기 
transforms = transforms.Compose([transforms.Resize((32,32)),
                                 transforms.ToTensor()])


mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms,
                          download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms,
                         download=True)

In [50]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last = True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last = True)

In [51]:
class LeNet5(torch.nn.Module):
    def __init__(self):
      super(LeNet5,self).__init__()

      
      self.c1 = torch.nn.Conv2d(1,6,kernel_size=5,stride=1,padding=0)
      self.x1 = torch.nn.Tanh()
      self.s2 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

      self.c3 = torch.nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0)
      self.x3 = torch.nn.Tanh()
      self.s4 = torch.nn.AvgPool2d(kernel_size=2,stride=2)

      #self.fl = torch.nn.Faltten()

      self.f5 = torch.nn.Linear(16*5*5,120,bias=True)
      self.x5 = torch.nn.Tanh()

      self.f6 = torch.nn.Linear(120,84,bias=True)
      self.x6 = torch.nn.Tanh()

      self.output = torch.nn.Linear(84,10,bias=True)

    def forward(self,x):
      out = self.c1(x)
      out = self.x1(out)
      out = self.s2(out)
      out = self.c3(out)
      out = self.x3(out)
      out = self.s4(out)
      out = out.view(out.size(0),-1)
      out = self.f5(out)
      out = self.x5(out)
      out = self.f6(out)
      out = self.x6(out)
      out = self.output(out)
      return out

In [52]:
model = LeNet5().to(device)


In [53]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [54]:
total_batch = len(train_loader)
print('Learning started. It takes sometime')
for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))
print('Learning Finished!')

Learning started. It takes sometime
[Epoch :    1] cost = 0.330930531
[Epoch :    2] cost = 0.0942099616
[Epoch :    3] cost = 0.061307881
[Epoch :    4] cost = 0.0468059666
[Epoch :    5] cost = 0.0385779291
[Epoch :    6] cost = 0.028185457
[Epoch :    7] cost = 0.0236899052
[Epoch :    8] cost = 0.0183655974
[Epoch :    9] cost = 0.0144372219
[Epoch :   10] cost = 0.0116448803
[Epoch :   11] cost = 0.00952292792
[Epoch :   12] cost = 0.00803068094
[Epoch :   13] cost = 0.00619505579
[Epoch :   14] cost = 0.00483162375
[Epoch :   15] cost = 0.00395467179
[Epoch :   16] cost = 0.00310695521
[Epoch :   17] cost = 0.00304762973
[Epoch :   18] cost = 0.00210876367
[Epoch :   19] cost = 0.00163724937
[Epoch :   20] cost = 0.00139037066
[Epoch :   21] cost = 0.00118700753
[Epoch :   22] cost = 0.000951472612
[Epoch :   23] cost = 0.000862216461
[Epoch :   24] cost = 0.000813177088
[Epoch :   25] cost = 0.000683657767
[Epoch :   26] cost = 0.000621284882
[Epoch :   27] cost = 0.00056041358


In [55]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
    model.eval()

    for X,Y in test_loader:
      X = X.to(device)
      Y = Y.to(device)

      prediction = model(X)
      correct_prediction = torch.argmax(prediction,1) == Y
      accuracy += correct_prediction.float().mean()
print('Accuracy:',accuracy.item()/total_batch)

Accuracy: 0.9903846153846154


In [ ]:
#0.99